In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import glob, os
from IPython.display import display

In [ ]:
# pre-processing data
def format(file):
    # skip reading all ball-related data
    df = pd.read_csv(file, sep=',', skiprows=1, warn_bad_lines=False, error_bad_lines=False, index_col=None, mangle_dupe_cols=True) 
    df = df[df["info"] != "ball"] 
    # first column not required
    df.drop(columns=["info"], inplace=True)
    # df = pd.read_csv("C:\Users\alizo\OneDrive - University of Waterloo\Desktop\Ali\Universities\Data Science\Cricket ML\Source\1211660.csv")
    # data format is transposed
    df = df.transpose(copy=True)
    headers = df.iloc[0]
    df = df[1:]
    df.columns = headers
    # multiple columns with same names
    df.columns = pd.io.parsers.ParserBase({'names':df.columns})._maybe_dedup_names(df.columns)
    df.reset_index(inplace=True)
    df.rename(columns={ df.columns[0]: "team1", df.columns[1]: "team2", "umpire" : "umpire1", "umpire.1" : "umpire2"}, inplace=True)
    return df



In [ ]:
df = pd.DataFrame()
path = "C:/Users/alizo/OneDrive - University of Waterloo/Desktop/Ali/Universities/Data Science/Cricket ML/Source"
all_files = glob.glob(os.path.join(path, "*.csv"))
for i in all_files:
    new_df = format(i)
    df = pd.concat([df, new_df], axis=0, ignore_index=True)    

df.info()
df.head()

In [ ]:
# city dictionary
city_map = {'Dubai International Cricket Stadium' : 'Dubai',
            'Sharjah Cricket Stadium' : 'Sharjah',
            'Gaddafi Stadium' : 'Lahore',
            'National Stadium' : 'Karachi',
            'Sheikh Zayed Stadium' : 'Abu Dhabi',
            'Multan Cricket Stadium' : 'Multan',
            'Rawalpindi Cricket Stadium' : 'Rawalpindi'}
df['city'] = df.apply(lambda row: city_map[row['venue']], axis=1)
# df.head()

In [ ]:
# df.method.unique()
# df.outcome.unique()
# df[pd.isnull(df['winner'])]


In [ ]:
# tie games
df['winner'] = df.apply(
    lambda row: row['eliminator'] if pd.isnull(row['winner']) and pd.notnull(row['eliminator']) else row['winner'],
    axis=1
)
# games decided by D/L (washed out) 
df['outcome'] = df.apply(
    lambda row: "D/L" if pd.isnull(row['outcome']) and (row['method'] == "D/L") else row['outcome'],
    axis=1
)
df.outcome.fillna("Result", inplace=True)
df.drop(columns=["match_number", "eliminator", "method"], inplace=True)

In [ ]:
df.info()
df.describe()